In [1]:
import pandas as pd

from ddeserts.annotate import add_all_stat_columns
from ddeserts.annotate import with_columns_sorted
from ddeserts.load import load_cvap_data

In [2]:
df = load_cvap_data('State')

In [4]:
# add extra columns
add_all_stat_columns(df)
df = with_columns_sorted(df)
df

geoid               geoname   adu_est  adu_moe   cit_est  \
table line                                                                 
State 1     04000US01               Alabama   3779875      543   4770940   
      14    04000US02                Alaska    552675      202    712455   
      27    04000US04               Arizona   5414955      227   6519390   
      40    04000US05              Arkansas   2295100      584   2903080   
      53    04000US06            California  30261350      462  34187375   
      66    04000US08              Colorado   4349345      496   5298855   
      79    04000US09           Connecticut   2831240      236   3326750   
      92    04000US10              Delaware    753565      118    909575   
      105   04000US11  District of Columbia    568755       91    639585   
      118   04000US12               Florida  16719175      616  18993615   
      131   04000US13               Georgia   7898605     1097   9809250   
      144   04000US15                Hawaii   1117455        0   1313265   
      157   04000US16                 Idaho   1276605      302   1656805   
      170   04000US17              Illinois   9879105      621  11884945   
      183   04000US18               Indiana   5093210      641   6455225   
      196   04000US19                  Iowa   2410770      499   3039680   
      209   04000US20                Kansas   2199580      484   2784630   
      222   04000US21              Kentucky   3439745      877   4345650   
      235   04000US22             Louisiana   3561010      631   4549895   
      248   04000US23                 Maine   1082995      158   1314655   
      261   04000US24              Maryland   4677165      215   5577240   
      274   04000US25         Massachusetts   5479295      231   6314695   
      287   04000US26              Michigan   7787385      344   9639085   
      300   04000US27             Minnesota   4267530      328   5335310   
      313   04000US28           Mississippi   2270925      448   2941850   
      326   04000US29              Missouri   4723300      723   5971680   
      339   04000US30               Montana    822265      404   1040280   
      352   04000US31              Nebraska   1440465      333   1830380   
      365   04000US32                Nevada   2291340      212   2681290   
      378   04000US33         New Hampshire   1088160      153   1311780   
      391   04000US34            New Jersey   6916980      383   8011800   
      404   04000US35            New Mexico   1603980      360   1976245   
      417   04000US36              New York  15463820      352  17686675   
      430   04000US37        North Carolina   7968260      697   9766885   
      443   04000US38          North Dakota    580975      321    738305   
      456   04000US39                  Ohio   9050385      534  11397175   
      469   04000US40              Oklahoma   2975905      425   3782285   
      482   04000US41                Oregon   3261860      334   3905205   
      495   04000US42          Pennsylvania  10129140      403  12379320   
      508   04000US44          Rhode Island    849885       99    994510   
      521   04000US45        South Carolina   3918305      365   4873320   
      534   04000US46          South Dakota    656530      415    850795   
      547   04000US47             Tennessee   5204435      826   6499470   
      560   04000US48                 Texas  20922410     1090  25253945   
      573   04000US49                  Utah   2173265      324   2938730   
      586   04000US50               Vermont    507660      150    610980   
      599   04000US51              Virginia   6588765      957   7953315   
      612   04000US53            Washington   5760560      447   6850930   
      625   04000US54         West Virginia   1447865      579   1802675   
      638   04000US55             Wisconsin   4509815      376   5633105   
      651   04000US56               Wyoming    444655      456  

In [ ]:
df.to_csv('output/states.csv')